In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
data1 = pd.read_csv('vertebral_2C.csv')

In [3]:
data1['Label'] = data1['Class Label'].apply(lambda x: 1 if x == 'AB' else -1)

In [4]:
data1.drop(['Class Label'],axis=1,inplace=True)
T = data1['Label']

In [5]:
#meanPositive = data1[data1['Label'] == 1].drop(['Label'],axis=1).mean()
#meanNegative = data1[data1['Label'] == 0].drop(['Label'],axis=1).mean()

In [6]:
#covPositive = data1[data1['Label'] == 1].drop(['Label'],axis=1).cov()
#covNegative = data1[data1['Label'] == 0].drop(['Label'],axis=1).cov()

In [7]:
#Positivedet = np.linalg.det(covPositive)
#print(Positivedet)
#negativedet = np.linalg.det(covNegative)
#print(negativedet)

In [8]:
#nneg = data1[data1['Label'] == 0]['Label'].count()
#print("negetive count",nneg)
#npos = data1[data1['Label'] == 1]['Label'].count()
#print("positive count",npos)

In [9]:
def model(data):
    meanPositive = data[data['Label'] == 1].drop(['Label'],axis=1).mean()
    meanNegative = data[data['Label'] == -1].drop(['Label'],axis=1).mean()
    covPositive = data[data['Label'] == 1].drop(['Label'],axis=1).cov()
    covNegative = data[data['Label'] == -1].drop(['Label'],axis=1).cov()
    Positivedet = np.linalg.det(covPositive)
    negativedet = np.linalg.det(covNegative)
    nneg = data[data['Label'] == -1]['Label'].count()
    npos = data[data['Label'] == 1]['Label'].count()
    return meanPositive,meanNegative,covPositive,covNegative,Positivedet,negativedet,nneg,npos

In [10]:
import math
def pdf(mean, coMatrix, coDet, x):
    var = float(coDet)**0.5
    denom = (2*math.pi*var)
    xsubmean = np.matrix(x-mean)
    invcovariance = np.linalg.inv(coMatrix)
    meanInvProduct = xsubmean* invcovariance
    xsubMeanTranspose= np.transpose([np.array(x-mean)])
    num = math.exp(-1 *meanInvProduct *xsubMeanTranspose /2 )
    #print("pdf ", num/denom)
    return num/denom

def positveBaysian(meanPositive,meanNegative,covPositive,covNegative,Positivedet,negativedet,x,npos,nneg):
    pdfPositive = pdf(meanPositive,covPositive,Positivedet,x)
    pdfNegative = pdf(meanNegative,covNegative,negativedet,x)
    ProbPositive = npos * pdfPositive / (nneg * pdfNegative + npos* pdfPositive  )
    return ProbPositive

def negativeBaysian(meanPositive,meanNegative,covPositive,covNegative,Positivedet,negativedet,x,npos,nneg):
    pdfPositive = pdf(meanPositive,covPositive,Positivedet,x)
    pdfNegative = pdf(meanNegative,covNegative,negativedet,x)
    ProbNegative = nneg* pdfNegative/ (nneg * pdfNegative + npos * pdfPositive  )
    return ProbNegative

#pr5baysian = positveBaysian(meanPositive,meanNegative,covPositive,covNegative,Positivedet,negativedet,pp[0],pp[1])

In [11]:
scaler = StandardScaler()
#X  = scaler.fit_transform(data1.drop(['Label'],axis=1))
#X = data1.drop(['Label'],axis=1).as_matrix()

In [12]:
cols = data1.columns
cols[0]

'pelvic incidence'

In [13]:
#X1 = pd.DataFrame(X,columns=cols[:-1])

In [14]:
#calculate baysian accuracy for whole data
def baysianWholeData(data):
    meanPositive,meanNegative,covPositive,covNegative,Positivedet,negativedet,nneg,npos = model(data)
    TNbaysian = 0
    TPbaysian = 0
    FPbaysian=0
    FNbaysian=0
    X = data.drop(['Label'],axis=1).as_matrix()
    T = data['Label']
    for i in range(0,len(data)):
        x = X[i]
        y = T[i]
        if(y == -1):
            prneg = negativeBaysian(meanPositive,meanNegative,covPositive,covNegative,Positivedet,negativedet,x,npos,nneg)
            if( prneg > 0.5):
                TNbaysian = TNbaysian+1
            else:
                FPbaysian = FPbaysian + 1
        else:
            prpos = positveBaysian (meanPositive,meanNegative,covPositive,covNegative,Positivedet,negativedet,x,npos,nneg)
            if(prpos > 0.5):
                #print("positive 5",pr5)
                TPbaysian = TPbaysian + 1
            else:
                FNbaysian = FNbaysian + 1
    baysianAccuracy = ( TPbaysian + TNbaysian )/ len(data)
    sensitivity = TPbaysian / (TPbaysian + FNbaysian)
    specificity = TNbaysian / ( TNbaysian + FPbaysian)
    PPV = TPbaysian / (TPbaysian + FPbaysian)
    print("Accuracy" ,baysianAccuracy )
    print("Sensitivity",sensitivity )
    print("Specificity",specificity)
    print("PPv",PPV )

baysianWholeData(data1)

Accuracy 0.8483870967741935
Sensitivity 0.7952380952380952
Specificity 0.96
PPv 0.9766081871345029


In [15]:
X_train, X_test, y_train, y_test = train_test_split(data1.drop(['Label'],axis=1), data1['Label'], test_size=0.33, random_state=42)

In [16]:
#baysian accuracy for train/test data
def baysianPartialData(data):
    X_train, X_test, y_train, y_test = train_test_split(data.drop(['Label'],axis=1), data['Label'], test_size=0.33, random_state=42)
    meanPositive,meanNegative,covPositive,covNegative,Positivedet,negativedet,nneg,npos = model(pd.concat([X_train,y_test],axis=1))
    TNbaysian = 0
    TPbaysian = 0
    X = X_test.as_matrix()
    T = y_test.as_matrix()
    for i in range(0,len(X_test)):
        x = X[i]
        y = T[i]
        if(y == -1):
            prneg = negativeBaysian(meanPositive,meanNegative,covPositive,covNegative,Positivedet,negativedet,x,npos,nneg)
            if( prneg > 0.5):
                TNbaysian = TNbaysian+1
        else:
            prpos = positveBaysian (meanPositive,meanNegative,covPositive,covNegative,Positivedet,negativedet,x,npos,nneg)
            if(prpos > 0.5):
                #print("positive 5",pr5)
                TPbaysian = TPbaysian + 1
    baysianAccuracy = ( TPbaysian + TNbaysian )/ len(X_test)
    return baysianAccuracy

print("baysian accuracy for partial data ",baysianPartialData(data1))

baysian accuracy for partial data  0.0


/Users/d062804/anaconda/lib/python3.6/site-packages/numpy/linalg/linalg.py:1804: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
